# Piramide de vacinação brasileira

Fontes:

1. população brasileira - [Projeção da população do Brasil e das Unidades da Federação - IBGE](https://www.ibge.gov.br/apps/populacao/projecao//index.html)
2. população vacinada - [Registros de Vacinação COVID19 - OpenDataSUS](https://opendatasus.saude.gov.br/dataset/covid-19-vacinacao/resource/ef3bd0b8-b605-474b-9ae5-c97390c197a8)

**ANTES DE RODAR O CÓDIGO**

- Baixe o ```.csv``` completo com os dados de vacinação no Brasil e salve o arquivo na mesma pasta aonde o jupyter notebook está.

- No local de date coloque a data de corte para até onde você irá pegar dados. Se deseja pegar dados de vacinação até o dia 01/06/2021, por exemplo, coloque a data como 02/06/2021.

- O ```.csv``` é grande demais para ser aberto sem estourar a RAM do meu computador (se o seu aguenta parabéns, você tem um ótimo computador). Portanto, este código faz um loop que abre ele em partes de 7 milhões de linhas (se quiser mudar isso, mude a variável ```rows_n``` para o valor que queira). Logo, antes de rodar o código, você precisa checar quantas pessoas já foram vacinadas até a data que deseja. Por exemplo, se já 140 milhões de pessoas já foram vacinadas até a data que você quer, o primeiro subloop deve ir até 140 milhões/7 milhões = 20

- Para checar quantas pessoas já foram vacinadas, recomendo acessar [ourworldindata.org/vaccination_brazil](ourworldindata.org/vaccination_brazil).

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import scipy.stats as st
import os
import datetime
import time

In [ ]:
dates = ['2021-01-24', '2021-01-31', '2021-02-07', '2021-02-14', '2021-02-21',
         '2021-02-28', '2021-03-07', '2021-03-14', '2021-03-21', '2021-03-28',
         '2021-04-04', '2021-04-11', '2021-04-18', '2021-04-25', '2021-05-02',
         '2021-05-09', '2021-05-16', '2021-05-23', '2021-05-30', '2021-06-06',
         '2021-06-13', '2021-06-20', '2021-06-27', '2021-07-04', '2021-07-11',
         '2021-07-18', '2021-07-25', '2021-08-01', '2021-08-08', '2021-08-15',
         '2021-08-22', '2021-08-29', '2021-09-05', '2021-09-12', '2021-09-19',
         '2021-09-26']
dates_pd = pd.DataFrame({'dates': dates})
dates_pd['dates'] = pd.to_datetime(dates_pd['dates'], format = '%Y-%m-%d').dt.tz_localize(None)

In [ ]:
pop_br = 212981000
cols = ['paciente_idade', 'vacina_dataaplicacao', 'vacina_descricao_dose',
        'paciente_enumsexobiologico']

man = [0.0356,0.0354,0.0357,0.0380,0.0411,0.0401,0.0403,0.0394,0.0358,0.0311,0.0285,0.0251,0.0205,
       0.0158,0.0113,0.0073,0.0045,0.0022,0.0013]
woman = [0.0340,0.0338,0.0342,0.0366,0.0402,0.0401,0.0410,0.0410,0.0379,0.0334,0.0311,0.0281,0.0238,
         0.0189,0.0142,0.0098,0.0067,0.0037,0.0026]

br = pd.DataFrame({'Age': ['0-4','5-9','10-14','15-19','20-24','25-29','30-34','35-39','40-44',
                           '45-49','50-54','55-59','60-64','65-69','70-74','75-79','80-84','85-89','90+'],
                   'Male': np.array(man)*pop_br,
                   'Female': np.array(woman)*(-pop_br)})
AgeClass = ['90+','85-89','80-84','75-79','70-74','65-69','60-64','55-59','50-54','45-49','40-44',
            '35-39','30-34','25-29','20-24','15-19','10-14','5-9','0-4']

In [ ]:
rows_n = 7000000
size = 34

In [ ]:
start = time.time()

idx = -1
for date in dates:
    idx += 1
    dat = date
    print(f'Threshold date: {dat}')
    male_first = []
    female_first = []
    male_second = []
    female_second = []

    for i in range(size):
        elapsed_time = (time.time() - start)/60
        print('\r', f'Section: {i} of {size}. Progress: {(100*(i)/size):.2f}%. Execution time: {elapsed_time:.2f} min', end = '')
        if i == 0:
            vac = pd.read_csv('Vacinacao_COVID_completo_Brasil.csv',
                            error_bad_lines=False, nrows=rows_n, delimiter = ';', usecols = cols)
        elif 0 < i < size-1:
            vac = pd.read_csv('Vacinacao_COVID_completo_Brasil.csv',
                            error_bad_lines=False, skiprows=range(1, i*rows_n) , nrows=rows_n,
                            delimiter = ';', usecols = cols)
        elif i == size-1:
            vac = pd.read_csv('Vacinacao_COVID_completo_Brasil.csv',
                            error_bad_lines=False, skiprows=range(1, i*rows_n) , nrows=rows_n,
                            delimiter = ';')
        vac['vacina_dataaplicacao'] = pd.to_datetime(vac['vacina_dataaplicacao'], format = '%Y-%m-%d').dt.tz_localize(None)
        vac = vac.loc[vac['vacina_dataaplicacao'] < dates_pd['dates'].values[idx]] #Change here if there are more than one dates
        # if len(vac['vacina_dataaplicacao']) > 0:
        #   print('\r', f"Most recent date in this section: {vac.sort_values('vacina_dataaplicacao').reset_index(drop=True)['vacina_dataaplicacao'][len(vac['vacina_dataaplicacao'])-1]}", end = '')
        vacina_1_dose = vac.loc[vac['vacina_descricao_dose'] == '1ª\xa0Dose']
        vacina_2_dose = vac.loc[vac['vacina_descricao_dose'] == '2ª\xa0Dose']
        vacina_dose_unica = vac.loc[vac['vacina_descricao_dose'] == 'Dose\xa0']
        for j in range(19):
            x = vacina_1_dose.loc[vacina_1_dose['paciente_idade'] >= j*5]
            x = x.loc[x['paciente_idade'] <= j*5 + 4]
            x2 = vacina_2_dose.loc[vacina_2_dose['paciente_idade'] >= j*5]
            x2 = x2.loc[x2['paciente_idade'] <= j*5 + 4]
            x3 = vacina_dose_unica.loc[vacina_dose_unica['paciente_idade'] >= j*5]
            x3 = x3.loc[x3['paciente_idade'] <= j*5 + 4]
            if j == 18:
                x = vacina_1_dose.loc[vacina_1_dose['paciente_idade'] >= j*5]
                x2 = vacina_2_dose.loc[vacina_2_dose['paciente_idade'] >= j*5]
                x3 = vacina_dose_unica.loc[vacina_dose_unica['paciente_idade'] >= j*5]
            m = len(x.loc[x['paciente_enumsexobiologico'] == 'M'])
            m2 = len(x2.loc[x2['paciente_enumsexobiologico'] == 'M'])
            m3 = len(x3.loc[x3['paciente_enumsexobiologico'] == 'M'])
            male_first.append(m)
            male_second.append(m2+m3)
            f = len(x.loc[x['paciente_enumsexobiologico'] == 'F'])
            f2 = len(x2.loc[x2['paciente_enumsexobiologico'] == 'F'])
            f3 = len(x3.loc[x3['paciente_enumsexobiologico'] == 'F'])
            female_first.append(-f)
            female_second.append(-f2-f3)
        elapsed_time = (time.time() - start)/60
        print('\r', f'Section: {i+1} of {size}. Progress: {(100*(i+1)/size):.2f}%. Execution time: {elapsed_time:.2f} min', end = '')
    print(' ')
    #male_first = np.array(male_first)
    #female_first = np.array(female_first)
    male_final_1 = np.zeros(19)
    female_final_1 =np.zeros(19)
    #print(male_first.shape)
    #male_second = np.array(male_second)
    #female_second = np.array(female_second)
    male_final_2 = np.zeros(19)
    female_final_2 =np.zeros(19)

    for i in range(int(len(male_first)/19)):
        male_final_1 += male_first[i*19:i*19+19]
        female_final_1 += female_first[i*19:i*19+19]
        male_final_2 += male_second[i*19:i*19+19]
        female_final_2 += female_second[i*19:i*19+19]

    print(male_final_1.shape)
    print(female_final_1.shape)

    df = pd.DataFrame({'Age': ['0-4','5-9','10-14','15-19','20-24','25-29','30-34','35-39','40-44','45-49','50-54','55-59','60-64','65-69','70-74','75-79','80-84','85-89','90+'],
                   'Male_1dose': male_final_1,
                   'Female_1dose': female_final_1,
                   'Male_2dose': male_final_2,
                   'Female_2dose': female_final_2})
    print(df.head(5))
    plt.figure(figsize=(10,8))
    bar_plot = sns.barplot(x='Male', y='Age', data=br, order=AgeClass, color = 'blue', alpha = 0.4, label = 'Homens população')
    bar_plot = sns.barplot(x='Female', y='Age', data=br, order=AgeClass, color = 'red', alpha = 0.4, label = 'Mulheres população')

    bar_plot = sns.barplot(x='Male_1dose', y='Age', data=df, order=AgeClass, color = 'blue', alpha = 0.7, label = 'Homens vacinados 1ª dose')
    bar_plot = sns.barplot(x='Female_1dose', y='Age', data=df, order=AgeClass, color = 'red', alpha = 0.7, label = 'Mulheres vacinadas 1ª dose')

    bar_plot = sns.barplot(x='Male_2dose', y='Age', data=df, order=AgeClass, color = 'darkblue', label = 'Homens vacinados 2ª dose')
    bar_plot = sns.barplot(x='Female_2dose', y='Age', data=df, order=AgeClass, color = 'darkred', label = 'Mulheres vacinados 2ª dose')

    plt.ylabel('Grupo Etário', fontsize = 14)
    plt.xlabel('População (milhões)', fontsize = 14)
    plt.title(f'Pirâmide de vacinação ({dat})', fontsize = 18)
    plt.xticks([-10e6, -7.5e6, -5e6, -2.5e6, 0, 2.5e6, 5e6, 7.5e6, 10e6],
            labels = ['10','7.5','5','2.5','0','2.5','5','7.5','10'])
    plt.legend(title = 'Populações', bbox_to_anchor = (1.01,1), fontsize = 11)
    plt.savefig(f'vaccination_pyramid_{dat}.png', bbox_inches = 'tight', dpi = 300)
    plt.close()